In [1]:
y = dbutils.widgets.get("outputFromLookup")
print(y)

# dbutils.widgets.text("inputpath", "","")
# dbutils.widgets.get("inputpath")
# datafromDb = getArgument("inputpath")
# inputdata = js.loads(datafromDb)
# inputdata = inputdata["Response"]
# # Importing the data set
# dataset = pd.read_csv(StringIO(inputdata),header=0,delimiter=',',skipinitialspace=True)

In [2]:
import ast
import pandas as pd
#Converting str to dictionary
d = ast.literal_eval(y) 
valueList = []
for i in range(0, len(d['value']), 1):
  valueList.append(d['value'][i])
df_pd = pd.DataFrame(valueList)  
print(df_pd)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

In [4]:
x = df_pd.iloc[:, 0:3]
y = df_pd.iloc[:, 3:4]
Gender = pd.get_dummies(x['Gender'])
x = x.drop('Gender', axis=1)
x = pd.concat([x, Gender], axis=1)
scX = StandardScaler()
x = scX.fit_transform(x)
print(x)

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

classification = DecisionTreeClassifier()
classification.fit(x_train, y_train)

y_pred = classification.predict(x_test)

# x_test = x_test.reset_index(drop=True)
accuracy = r2_score(y_test, y_pred)
print("Accuracy", accuracy)

In [6]:
# upload file into blob storage
from azure.storage.blob import *
import pickle

storage_account_name = 'str1wk6'
storage_account_access_key = 'OnTwDpJa14kT8Xi9rY36r9kKeUqNXJdCm1D4RLl9LgUUGBFdmKjQoWhpn2dljkfDAhuDXEprXyB7Ld0eFW+upQ=='
containerName = 'cnt1'
localFileName = '/opt/DecisionTreeClassification.pkl'
pickle.dump(classification, open(localFileName, 'wb'))

blob_storage = BlockBlobService(account_name=storage_account_name, account_key=storage_account_access_key)
blob_storage.create_blob_from_path(containerName, 'DecisionTreeClassification.pkl', localFileName)